<a href="https://colab.research.google.com/github/vivivicdjdk/machine/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#교차 검증과 그리드 서치

일반적인 모델 생성 방법

1. 훈련 세트로 모델을 생성

  * 하이퍼 파라미터 조절

2. 테스트 세트로 모델 평가

이러한 과정을 반복하다보면 결국 모델이 훈련 세트 뿐만 아니라 테스트 세트에도 맞추어짐

검증 세트(validation set)

모델을 만들고 나서 테스트 세트는 딱 한 번 사용되는 것이 좋음

따라서 추가로 검증 세트를 또 나눔

대략 훈련세트 60%, 검증 세트 20%, 테스트 세트 20%

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [4]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트와 검증 세트의 평가 점수를 이용하여 최적화

최적화 완료 후 테스트 세트로 모델을 최종 평가

In [6]:
#최적화
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=3, random_state=42)
#max_depth
#min_impurity_decrease
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.8482078421938898
0.8490384615384615


In [5]:
print(dt.score(test_input, test_target))

0.8569230769230769


검증 세트의 문제점?

검증 세트로 인해 훈련 세트가 줄어듬

보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐

검증 세트를 너무 조금 뗴어 놓으면 검증 점수가 불안정

이러한 문제를 보완할 방법이 필요

교차검증(cross validation)

검증 세트를 떼어 내어 평가하는 과정을 여러번 반복

예를 들어, 3-폴드 교차 검증은 훈련 세트를 3부분으로 나누어 그 중 하나를 교대로 검증 세트로 설정

즉, 훈련세트를 1,2,3폴드로 나눈다면 1,2폴드는 훈련에 사용하고 3폴드는 검증에 사용한 후에 1,3은 훈련, 2는 검증, 마지막으로 2,3은 훈련, 1은 검증으로 사용

검증 점수의 평균을 최종 점수로 정함

보통은 5나 10-폴드 교차 검증 사용

cross_validate()는 기본적으로 5-폴드 교차검증임

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00547099, 0.01026654, 0.00520706, 0.00512838, 0.00508118]), 'score_time': array([0.00143814, 0.00446844, 0.00138831, 0.00565434, 0.00139046]), 'test_score': array([0.84230769, 0.83365385, 0.84504331, 0.8373436 , 0.8479307 ])}


교차 검증의 최종 점수

이름은 test_score이지만 테스트 점수가 아닌 검증 점수

In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.8412558303102096


주의

cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음

만약 train_test_split()을 하지 않았다면 분할기를 지정해야함

cross_validate()는 회귀 모델일 경우 KFild 분할기를 사용하고 분류 모델일 경우 StratifiedKFold를 사용

섞지 않는 것이 디폴트이므로 따로 설정해줘야

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(StratifiedKFold())
print(scores)
print(np.mean(scores['test_score']))

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
{'fit_time': array([0.00525451, 0.00474405, 0.00501132, 0.00522089, 0.00520968]), 'score_time': array([0.00132728, 0.00162244, 0.00144672, 0.00159883, 0.00156879]), 'test_score': array([0.84230769, 0.83365385, 0.84504331, 0.8373436 , 0.8479307 ])}
0.8412558303102096


훈련 세트를 먼저 섞은 후 10-폴드 교차 검증을 수

In [11]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
print(splitter)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(scores)
print(np.mean(scores['test_score']))

StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
{'fit_time': array([0.00598693, 0.0057261 , 0.00550008, 0.00531316, 0.00527048,
       0.00549102, 0.00585699, 0.00596261, 0.0054369 , 0.0054276 ]), 'score_time': array([0.00162935, 0.00189471, 0.00126934, 0.00133061, 0.00199556,
       0.00140476, 0.00138831, 0.00141931, 0.00134611, 0.00165248]), 'test_score': array([0.82307692, 0.86153846, 0.80769231, 0.83653846, 0.83461538,
       0.83653846, 0.85      , 0.8150289 , 0.83815029, 0.83236994])}
0.8335549132947977


cross_validate는 자체적으로 모델을 훈련시키기는 하나

따라서 훈련(fit)필수

교차 검증 아이디어를 활용하여 하이퍼파라미터 튜닝을 수행

매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을~



##하이퍼파라미터 튜닝

In [12]:
from sklearn.model_selection import GridSearchCV #CV(CrossValidate 교차검증을 사용함)

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [13]:
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


검증 점수가 가장 높은 매개변수 조합의 모델은 gs.best_estimator_속성에 저장됨

In [14]:
print(gs.best_estimator_)
print(gs.best_params_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수

In [15]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


넘파이 argmax()함수를 사용하여 가장 큰 값의 인덱스를 추출

params 키에 저장된 매개변수 출력

In [16]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


그리드 서치를 이용한 최적화 과정 정리
  1. 탐색할 매개변수를 지정
  2. 훈련 세트

In [17]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10) #2,12,...,92
          }

dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수 확인

In [18]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


찾은 파라미터로 모델을 생성하고 테스트 점수 확인

In [19]:
dt = gs.best_estimator_ #자동으로 모델 생성
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.892053107562055
0.8615384615384616


##랜덤 서치

In [20]:
from scipy.stats import uniform, randint

rgen = randint(0, 10) #0~9 랜덤 정수
rgen.rvs(10)

array([0, 1, 7, 7, 7, 1, 5, 5, 2, 5])

10번의 샘플링했기 때문에 고르게 샘플링되는 것 같아 보이지 않음

그러나 샘플링 숫자를 늘리면 쉽게 확인 가능

1000번 샘플링해서 각 숫자의 갯수 확인

In [21]:
np.unique(rgen.rvs(1000), return_counts=True) #unique한 값 출력 0은 112번 나옴

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([112, 108, 115,  72,  84, 108,  96, 105,  99, 101]))

uniform의 사용법도 동일

In [22]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.10889272, 0.08573289, 0.13803558, 0.9285086 , 0.11735636,
       0.12115177, 0.97487173, 0.41339401, 0.68985785, 0.88607606])

In [24]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV
#dt = DecisionTreeClassifier(radom_state=42)
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
#n_iter 반복횟수 지정 (이 범위 내에서 100번 선택하겠다.)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [25]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 이미 전체 훈련 세트(train_input, train_target)로 훈련되어 best_estimator_에 저장

In [26]:
dt = gs.best_estimator_
print(dt)
print(dt.score(test_input, test_target))

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.86


주의

교차검증, 그리드 서치, 랜덤 서치는 모델을 훈련하는 것이 아니라 검증하는 것임

즉, 적절한 하이퍼파라미터를 찾는데 사용하야함

찾은 하이퍼파라미터로 훈련(fit())을 하여 모델을 만들어야 함

랜덤서치 그리드 서치 검증세트 나눌 필요 없음

---

In [27]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7afa58d373a0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7afa59823f40>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7afa59823070>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7afa58d348b0>},
                   random_state=42)

In [28]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
